# 05. 벡터의 유사도(Vector Similarity)

- 문장이나 문서의 유사도를 구하는 작업은 자연어 처리의 주요 주제중 하나.
- 기계가 계산하는 문서의 유사도의 성능은 각 문서의 단어들을 어떤 방법으로 수치화하여 표현했는지(DTM, Word2Vec 등), 문서 간의 단어들의 차이를 어떤 방법(유클리드 거리, 코사인 유사도 등으로 계산했는지)

## 05-01 코사인 유사도(Cosine Similarity)
- BoW에 기반한 단어 표현 방법인 DTM, TF-IDF, 또는 뒤에서 배우게 될 Word2Vec등과 같이 단어를 수치화할 수 있는 방법을 이해했다면 코사인 유사도를 이용하여 문서의 유사도를 구하는게 가능.

### 05-01 코사인 유사도(Cosine Similarity)
- 두 벡터 간의 코사인 각도를 이용하여 구할 수 있는 두 벡터의 유사도.
- 두 벡터의 방향이 완전히 동일한 경우에는 1의 값을 갖는다.
- 90도의 각을 이루면 0
- 180도로 반대의 방향을 가지면 -1의 값을 갖게 된다.
- 즉, 결국 코사인 유사도는 -1 이상 1 이하의 값을 가지며 값이 1에 가까울수록 유사도가 높다고 판단할 수 있다.


In [1]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

doc1 = np.array([0,1,1,1])
doc2 = np.array([1,0,1,1])
doc3 = np.array([2,0,2,2])

print('문서 1과 문서2의 유사도 :',cos_sim(doc1, doc2))
print('문서 1과 문서3의 유사도 :',cos_sim(doc1, doc3))
print('문서 2와 문서3의 유사도 :',cos_sim(doc2, doc3))

문서 1과 문서2의 유사도 : 0.6666666666666667
문서 1과 문서3의 유사도 : 0.6666666666666667
문서 2와 문서3의 유사도 : 1.0000000000000002


- 문서3은 문서2에서 단지 모든 단어의 빈도수가 1씩 증가했다.
- 한 문서 내의 모든 단어의 빈도수가 동일하게 증가하는 경우에는 기존의 문서와 코사인 유사도의 값이 1이라는 것.
- 유사도 연산에 문서의 길이가 영향을 받는다. -> 유클리드보다 코사인 유사도가 해결책이 될 수 있다.
- 코사인 유사도는 유사도를 구할 때 벡터의 방향(패턴)에 초점을 두므로 코사인 유사도는 문서의 길이가 다른 상황에서 비교적 공정한 비교를 할 수 있도록 도와준다.

### 05-02 유사도를 이용한 추천 시스템 구현하기
- TF-IDF와 코사인 유사도만으로 영화의 줄거리에 기반해서 영화를 추천하는 추천시스템 구현

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

data = pd.read_csv('./data/movies_metadata.csv', low_memory=False)
data.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [4]:
# 훈련 데이터는 총 24개의 열을 갖고 있으나 책의 지면의 한계로 일부 생략

# 상위 2만개 샘플
data = data.head(20000)

# TF-IDF 연산할 때 데이터에 null값이 들어있으면 에러 발생.
# TF-IDF의 대상이 되는 data의 overview 열에 결측값에 해당하는 null값이 있는지 확인.

# overview 열에 존재하는 모든 결측값을 전부 카운트하여 출력
print('overview 열의 결측값의 수:',data['overview'].isnull().sum())

overview 열의 결측값의 수: 135


In [5]:
# 결측값을 빈 값으로 대체
data['overview'] = data['overview'].fillna('')

# Null값을 빈값으로 대체.
tfidf = TfidfVectorizer(stop_words = 'english')
tfidf_matrix = tfidf.fit_transform(data['overview'])
print('TF-IDF 행렬의 크기(shape) :',tfidf_matrix.shape)

C:\Users\user\AppData\Local\Temp\ipykernel_2756\752022930.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['overview'] = data['overview'].fillna('')


TF-IDF 행렬의 크기(shape) : (20000, 47487)


In [6]:
# TF-IDF 행렬의 크기는 20,000의 행을 갖고 47,847의 열을 갖는 행렬.
# 다시 말해 20,000개의 영화를 표현하기 위해 총 47,487개의 단어가 사용되었음을 의미.

cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print('코사인 유사도 연산 결과 :', cosine_sim.shape)

코사인 유사도 연산 결과 : (20000, 20000)


- 위는 20000개의 각 문서 벡터(영화 줄거리 벡터)와 자기 자신을 포함한 20000개의 문서 벡터간의 유사도가 기록된 행렬.
- 모든 20,000개의 상호 유사도가 기록되어져 있다.
- 기존 데이터프레임으로부터 영화의 타이틀을 key, 영화의 인덱스를 value로 하는 딕셔너리 title_to_index 생성

In [7]:
cosine_sim

array([[1.        , 0.01575748, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.01575748, 1.        , 0.04907345, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.04907345, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.08375766],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.08375766, 0.        ,
        1.        ]])

In [8]:
title_to_index = dict(zip(data['title'], data.index))

# 영화 제목 Father of the Bride Part II의 인덱스를 리턴
idx = title_to_index['Father of the Bride Part II']
print(idx)

4


In [9]:
# 선택한 영화의 제목을 입력하면 코사인 유사도를 통해 가장 overview가 유사한 10개의 영화를 찾아내는 함수.

def get_recommendations(title, cosine_sim = cosine_sim):
    # 선택한 영화의 타이틀로부터 해당 영화의 인덱스를 받아온다.
    idx = title_to_index[title]

    # 해당 영화와 모든 영화와의 유사도를 가져온다.
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 영화들을 정렬한다.
    sim_scores = sorted(sim_scores, key=lambda x:x[1], reverse = True)

    # 가장 유사한 10개의 영화를 받아온다.
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개의 영화의 인덱스를 얻는다.
    movie_indices = [idx[0] for idx in sim_scores]

    # 가장 유사한 10개의 영화 제목 리턴.
    return data['title'].iloc[movie_indices]


In [10]:
get_recommendations('The Dark Knight Rises')

12481                            The Dark Knight
150                               Batman Forever
1328                              Batman Returns
15511                 Batman: Under the Red Hood
585                                       Batman
9230          Batman Beyond: Return of the Joker
18035                           Batman: Year One
19792    Batman: The Dark Knight Returns, Part 1
3095                Batman: Mask of the Phantasm
10122                              Batman Begins
Name: title, dtype: object

In [11]:
# 가장 유사한 영화가 출력되는데, 영화 다크 나이트가 첫번째고, 그 외에도 전부 배트맨 영화를 찾아낸다.

### 05-02 여러가지 유사도 기법

### 1. 유클리드 거리
- 유용한 방법은 아니다.
- 하지만 여러 가지 방법을 이해하고, 시도해보는 것 자체만으로 다른 개념들을 이해할 때 도움이 되므로 의미가 있다.
- 여러 문서에 대해 유사도를 구하고자 유클리더 거리 공식을 사용한다는 것은, 앞서 본 2차원을 단어의 총 개수만큼의 차원으로 확장하는 것과 같다.

In [13]:
import numpy as np

def dist(x,y):
    return np.sqrt(np.sum((x-y)**2))

doc1 = np.array((2,3,0,1))
doc2 = np.array((1,2,3,1))
doc3 = np.array((2,1,2,2))
docQ = np.array((1,1,0,1))

print('문서1과 문서Q의 거리 :', dist(doc1, docQ))
print('문서2과 문서Q의 거리 :', dist(doc2, docQ))
print('문서3과 문서Q의 거리 :', dist(doc3, docQ))



문서1과 문서Q의 거리 : 2.23606797749979
문서2과 문서Q의 거리 : 3.1622776601683795
문서3과 문서Q의 거리 : 2.449489742783178


- 유클리더 거리의 값이 가장 작다는 것은 문서 간 거리가 가장 가깝다는 것을 의미.
- 문서1이 문서Q와 가장 유사

### 2. 자카드 유사도(Jaccard Similarity)
- A와 B 두개의 집합이 있을때, 합집합에서 교집합의 비율으르 구한다면 두 집합 A와 B의 유사도를 구할 수 있다는 것이 자카드 유사도의 아이디어.
- 자카드 유사도는 0과 1사이의 값을 가지며, 만약 두 집합이 동일하다면 1의 값을 갖고,
- 두 집합의 공통 원소가 없다면 0의 값을 갖는다.

In [14]:
doc1 = 'apple banana everyone like likey watch card holder'
doc2 = 'apple banana coupon passport love you'

# 토큰화
tokenized_doc1 = doc1.split()
tokenized_doc2 = doc2.split()

print('문서1 :',tokenized_doc1)
print('문서2 :',tokenized_doc2)

문서1 : ['apple', 'banana', 'everyone', 'like', 'likey', 'watch', 'card', 'holder']
문서2 : ['apple', 'banana', 'coupon', 'passport', 'love', 'you']


In [15]:
# 문서1과 문서2의 합집합을 구해보자.
union = set(tokenized_doc1).union(set(tokenized_doc2))
print('문서1과 문서2의 합집합 :',union)

문서1과 문서2의 합집합 : {'banana', 'holder', 'likey', 'everyone', 'love', 'coupon', 'like', 'passport', 'apple', 'card', 'watch', 'you'}


In [16]:
# 문서1과 문서2의 합집합의 단어의 총 개수는 12개.
# 이제 문서1과 문서2의 교집합을 구해보자.
intersection = set(tokenized_doc1).intersection(set(tokenized_doc2))
print('문서1과 문서2의 교집합 :', intersection)


문서1과 문서2의 교집합 : {'banana', 'apple'}


In [17]:
print('자카드 유사도 :', len(intersection)/len(union))

자카드 유사도 : 0.16666666666666666
